In [2]:
from IPython.core.display import display
from scipy.optimize import linprog
import numpy as np
import random
from statistics import mean


# Кейс ФОРЕВА №1

## Исходные данные

In [3]:
n_products = 500 # сколько продуктов мы умеем производить производить
n_resources = 200 # колько у нас типпов станков

# из базы данных получаем множество кортежей вида (№ ресурса

Будем считать, что мы знаем количество этапов производства каждого продукта. Генерируем матрицу



Из базы данных получаем множество кортежей от MES-системы вида

(№ Продукта, № ресурса, № этапа, Затраченное время), где номер этапа нужен для того, чтобы определить повторное
использование одного и того же ресурса в рамках производства одного процесса для рассчета среднего времени операации


Для простоты модели не стал класть эти данные в БД и подключать ее просто сгенерировал. для репрезентативности кода
предположим, что мы можем учитвать последние 10000000 операций и эта информация может нам помочь судить о структуре
производственного процесса
Будем считать, что за этот период, мы сделали все продукты компании хотя бы по разу. Чтобы не усложнять излишне и не
упираться в производительность десктопа



In [4]:
mes_data = [
   [
       random.randint(1, n_products),
       random.randint(1, n_resources),
       random.randint(1,2),
       random.randint(0,20)
   ] for _ in range(10000000)
]

Далее составим "Программу производства"

In [5]:
prod_program = [
    [
        [
            [] for k in range(10)
        ] for j in range(n_resources)
    ] for i in range(n_products)
]
dt_count = [0 for _ in range(n_resources)]
res_prod_program = []
for md in mes_data:
    prod_program[md[0]-1][md[1]-1][md[2]-1].append(md[3])


for p in range(n_products):
    pp = []
    for r in range(n_resources):
        sum = 0

        for s in prod_program[p][r]:

            if s == []:
                sum += 0
            else:
                sum += round(mean(s),0)
                # dt_count[r] += 1

        pp.append(int(sum))
    res_prod_program.append(pp)
# предположим, что у нас в наличии от 1 до 3 единиц каждого станка
res_count = [random.randint(1,3) for _ in range(n_resources)]


"""
Для упрощения кода будем считать, что среднее время переналадки станка мы получили из MES
Просто сгенерируем в пределах от 0 до 120 минут
"""
res_downtime = [random.randint(0,120) for i in range(n_resources)]

price = [random.randint(100,200) for _ in range(n_products)]
rm_costs = [random.randint(20,99) for _ in range(n_products)]
demand = [random.randint(0,100) for _ in range(n_products)]

А теперь попробуем все это рассчитать


Понятно, что из-за того, что выборка была сгенерирована синтетически изначально, на количество 10 000 000 будет +-
нормальное распределение, но все же

Исходя из нормального распределения данных, пришлось увеличить лимиты по времени до 10 лет


In [6]:
price = np.array(price)
rm_costs = np.array(rm_costs)
product_program = res_prod_program
demand = np.array(demand)
resource_count = np.array(res_count)
resource_downtime = np.array(res_downtime)
downtime_count= np.array(dt_count)
time_limit = 8 * 5 * 60 * 52 * 10                        # время работы производства (в минутах)
opex = 16000  *10                                   # операционные расходы
resource_plan = np.transpose(np.array(product_program))

profit = np.subtract(price,rm_costs)
resource_limits = np.multiply(resource_count, time_limit)
resource_downtime_total = np.multiply(resource_downtime,downtime_count)
resource_usage = np.sum(resource_plan, axis=1)

c = np.negative(profit)
A_ub = resource_plan
downtime_total =  np.multiply(resource_downtime,downtime_count)
b_ub = np.subtract(np.multiply(resource_count,time_limit),downtime_total)
bounds = [(0, x) for x in demand]

result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='simplex')

In [7]:
print(f'Оптимизированная прибыль: {-result.fun-opex}')
print(f'Производственный план:')
j = 0
for i in result.x:
    j+=1
    print(f'Продукт №{j}: {i}')



Оптимизированная прибыль: 2066869.0
Производственный план:
Продукт №1: 15.0
Продукт №2: 31.0
Продукт №3: 79.0
Продукт №4: 80.0
Продукт №5: 17.0
Продукт №6: 52.0
Продукт №7: 46.0
Продукт №8: 96.0
Продукт №9: 58.0
Продукт №10: 8.0
Продукт №11: 38.0
Продукт №12: 95.0
Продукт №13: 27.0
Продукт №14: 1.0
Продукт №15: 59.0
Продукт №16: 0.0
Продукт №17: 82.0
Продукт №18: 52.0
Продукт №19: 60.0
Продукт №20: 2.0
Продукт №21: 88.0
Продукт №22: 57.0
Продукт №23: 5.0
Продукт №24: 87.0
Продукт №25: 40.0
Продукт №26: 20.0
Продукт №27: 92.0
Продукт №28: 99.0
Продукт №29: 48.0
Продукт №30: 48.0
Продукт №31: 2.0
Продукт №32: 16.0
Продукт №33: 29.0
Продукт №34: 52.0
Продукт №35: 54.0
Продукт №36: 99.0
Продукт №37: 49.0
Продукт №38: 65.0
Продукт №39: 8.0
Продукт №40: 50.0
Продукт №41: 44.0
Продукт №42: 71.0
Продукт №43: 76.0
Продукт №44: 92.0
Продукт №45: 48.0
Продукт №46: 30.0
Продукт №47: 77.0
Продукт №48: 69.0
Продукт №49: 24.0
Продукт №50: 13.0
Продукт №51: 21.0
Продукт №52: 63.0
Продукт №53: 81.0
Про

А теперь вернемся к нашей задаче:


In [12]:


price = np.array([180, 285, 210])               # Стоимость продуктов
rm_costs = np.array([30+35, 35+30+30, 75])      # стоимость ресурсов
product_program =[
    [0, 24, 33, 20, 8],                         # Product A
    [34, 9, 15, 18, 17],                        # Product D
    [14, 15, 22, 27, 0],                        # Product F
]


demand = np.array([40,80,40])                   # рыночный спрос
resource_count = np.array([1,2,2,2,1])          # Имеющееся количество ресурсов
resource_downtime = np.array([15,120,60,20,0])  # Время на переналадку ресурса
res_demand = [3, 4, 6, 4, 2]
downtime_count = np.array([3, 4, 6, 4, 2])      # количествво переналадок каждого ресурса
time_limit = 8 * 5 * 60                         # время работы производства (в минутах)
opex = 16000                                    # операционные расходы
capex_costs = np.array([])                      # капитализируемые косты




## Рассчеты
Заставляем машинку складывать нули и единицы

In [14]:
resource_plan = np.transpose(np.array(product_program))

profit = np.subtract(price,rm_costs)

c = np.negative(profit)
A_ub = resource_plan
downtime_total =  np.multiply(resource_downtime,downtime_count)
b_ub = np.subtract(np.multiply(resource_count,time_limit),downtime_total)
bounds = [(0, x) for x in demand]

result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='simplex')
print(result)

     con: array([], dtype=float64)
     fun: -20030.882352941175
 message: 'Optimization terminated successfully.'
     nit: 10
   slack: array([   0.        , 2284.85294118, 1448.08823529, 1889.70588235,
       1182.5       ])
  status: 0
 success: True
       x: array([40.        , 52.79411765, 40.        ])


## Результаты рассчетов

In [10]:
print(f'Оптимизированная прибыль: {-result.fun-opex}')
print(f'Производственный план:')
j = 0
for i in result.x:
    j+=1
    print(f'Продукт №{j}: {i}')

Оптимизированная прибыль: 4030.8823529411748
Производственный план:
Продукт №1: 40.0
Продукт №2: 52.794117647058826
Продукт №3: 40.0


Не хуже, чем в экселе


А теперь чуть подготовимся:

In [21]:
base_result = {
    'title': 'Без изменний',
    'cash': -result.fun-opex,
    'plan': result.x
}


def get_downtime_count(res_count, res_demand):
    dt_count_ddd = [
        [0,0,2,3,4,5,6],
        [0,0,0,2,4,4,6],
        [0,0,0,0,2,4,6],
        [0,0,0,0,0,2,4],
        [0,0,0,0,0,0,2],
        [0,0,0,0,0,0,0]
    ]
    return dt_count_ddd[res_count][res_demand]

def solve(title, pp_delta=None, r_count_delta=None, demand_delta=None, delta_opex=None):
    t_product_program = product_program
    t_res_count = resource_count
    t_downtime_count = downtime_count
    t_demand = demand
    t_opex = opex
    if pp_delta:
        for i in range(len(pp_delta)):
            for j in range(len(t_product_program)):
                t_product_program[j][i] += pp_delta[i]
    if r_count_delta:
        t_res_count = np.add(t_res_count, np.array(r_count_delta))
        # учитываем возможное уменьшение переналадок
        t_downtime_count = np.array([get_downtime_count(t_res_count[i], res_demand[i]) for i in range(len(t_res_count))])

    if demand_delta:
        t_demand = np.add(t_demand,np.array(demand_delta))
    if delta_opex:
        t_opex += delta_opex

    resource_plan = np.transpose(np.array(t_product_program))
    profit = np.subtract(price,rm_costs)
    c = np.negative(profit)
    A_ub = resource_plan
    downtime_total =  np.multiply(resource_downtime,t_downtime_count)
    b_ub = np.subtract(np.multiply(resource_count,time_limit),downtime_total)
    bounds = [(0, x) for x in t_demand]

    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='simplex')
    return {
    'title': title,
    'cash': -result.fun-opex,
    'plan': result.x
    }

Предположим нам поступило 100 рацпредложений, которые мы должны оценить и выбрать лучшее
Все рацпредложения представляют собой  улучшение производительности ресурса Х, но влекут
за собой увеличение временных затрат на ресурсе У

In [22]:
rac = []

for _ in range(1000):
    res = [0,0,0,0,0]
    x = random.randint(0,4)
    y = random.randint(0,4)
    while x == y:
        y = random.randint(0,4)
    nx = random.randint(1,10)
    ny = random.randint(1,10)
    res[x] = nx
    res[y] = -ny
    rac.append(res)

print(rac)

[[0, -2, 0, 0, 4], [0, -9, 0, 0, 9], [0, -4, 4, 0, 0], [-7, 0, 2, 0, 0], [0, 0, -5, 8, 0], [0, 0, 0, -8, 3], [0, -8, 0, 0, 2], [0, -2, 0, 0, 1], [0, -4, 0, 0, 7], [0, -3, 0, 1, 0], [5, 0, 0, -3, 0], [0, 0, 0, -9, 6], [-8, 0, 0, 6, 0], [2, 0, -7, 0, 0], [4, -5, 0, 0, 0], [-2, 0, 7, 0, 0], [0, 8, -4, 0, 0], [0, 0, 4, 0, -1], [0, 2, -6, 0, 0], [4, -7, 0, 0, 0], [0, 0, 0, -5, 2], [2, 0, 0, -7, 0], [0, 6, 0, 0, -2], [0, 0, 6, -9, 0], [0, 0, -2, 2, 0], [5, 0, -5, 0, 0], [0, 0, -8, 10, 0], [0, 0, -6, 0, 8], [0, 0, -2, 5, 0], [9, 0, 0, 0, -10], [0, 0, 0, -6, 1], [0, 6, -9, 0, 0], [5, -4, 0, 0, 0], [0, 0, 1, -7, 0], [0, 0, 0, 7, -2], [-1, 0, 4, 0, 0], [0, 3, 0, 0, -5], [0, 0, 0, 9, -1], [0, 0, -7, 0, 4], [5, 0, 0, -1, 0], [-4, 0, 3, 0, 0], [-3, 0, 0, 0, 9], [2, 0, 0, -10, 0], [0, 0, 0, 2, -6], [6, 0, 0, 0, -6], [-1, 0, 0, 0, 7], [-5, 0, 5, 0, 0], [0, -3, 8, 0, 0], [-2, 0, 4, 0, 0], [0, 0, 8, 0, -6], [0, 0, 0, -6, 3], [0, 8, 0, -3, 0], [0, 0, -7, 1, 0], [0, 0, -2, 2, 0], [5, 0, -8, 0, 0], [0, 0,

Давайте их оценим:


In [23]:
rres = []
for rr in rac:
    rres.append(solve(rr.__repr__(),pp_delta=rr))

print(rres)

[{'title': '[0, -2, 0, 0, 4]', 'cash': 4030.8823529411748, 'plan': array([40.        , 52.79411765, 40.        ])}, {'title': '[0, -9, 0, 0, 9]', 'cash': 1052.8011204481809, 'plan': array([14.10364146, 52.79411765, 40.        ])}, {'title': '[0, -4, 4, 0, 0]', 'cash': 1052.8011204481809, 'plan': array([14.10364146, 52.79411765, 40.        ])}, {'title': '[-7, 0, 2, 0, 0]', 'cash': 1306.6666666666642, 'plan': array([ 0.        , 62.66666667, 40.        ])}, {'title': '[0, 0, -5, 8, 0]', 'cash': 1306.6666666666642, 'plan': array([ 0.        , 62.66666667, 40.        ])}, {'title': '[0, 0, 0, -8, 3]', 'cash': -466.66666666666424, 'plan': array([ 0.        , 53.33333333, 40.        ])}, {'title': '[0, -8, 0, 0, 2]', 'cash': -1480.0, 'plan': array([ 0., 48., 40.])}, {'title': '[0, -2, 0, 0, 1]', 'cash': -1944.4444444444434, 'plan': array([ 0.        , 45.55555556, 40.        ])}, {'title': '[0, -4, 0, 0, 7]', 'cash': -4590.697674418607, 'plan': array([ 0.        , 31.62790698, 40.        ])